In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Datasets

Mounted at /content/drive
/content/drive/MyDrive/Datasets


In [ ]:
import torch
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

class Logger:
    def __init__(self):
        self.data = dict()

    def log(self, k, v):
        if not k in self.data:
            self.data[k] = []
        v = self._to_np(v)
        self.data[k].append(v)

    def __getitem__(self, k):
        return self.data.get(k, [])

    def _to_np(self, v):
        if isinstance(v, torch.Tensor):
            with torch.no_grad():
                return v.cpu().numpy()
        if isinstance(v, list):
            return [self._to_np(v_) for v_ in v]
        return v

def plot(log):
    plt.figure()
    plt.semilogy(log['irm_penalty'])
    plt.semilogy(log['rex_penalty'])
    plt.legend(['irm penalty', 'rex penalty'])
    plt.figure()
    plt.plot(log['train_acc'], 'k')
    plt.plot(log['test_acc'], alpha=0.7)
    plt.legend(['train acc'] + ['test acc'] * len(log['test_acc'][0]))
    plt.figure()
    plt.plot(log['losses'], 'k', alpha=0.3)
    plt.title('losses')
    plt.ylabel('nll')
    plt.show()

def save(logs, name):
    data = dict()
    for k in logs[0].data.keys():
        data[k] = np.array([l[k] for l in logs])
        np.savetxt('%s_%s_mean.dat' % (name, k), data[k].mean(0))
        np.savetxt('%s_%s_std.dat' % (name, k), data[k].std(0))
    np.savez_compressed('%s.npz' % name, **data)

def print_stats(step, log):
    pretty_print(
        np.int32(step),
        np.mean(log['train_nll'][-50:]),
        np.mean(log['train_acc'][-50:]),
        np.mean(log['irm_penalty'][-50:]),
        np.mean(log['rex_penalty'][-50:]),
        np.mean(log['test_acc'][-50:])
        #*np.array(log['test_acc'][-50:]).mean(axis=0),
    )

def pretty_print(*values):
    col_width = 13
    def format_val(v):
        if not isinstance(v, str):
            v = np.array2string(v, precision=5, floatmode='fixed')
        return v.ljust(col_width)
    str_values = [format_val(v) for v in values]
    print("   ".join(str_values))

def print_env_info(train_envs, test_envs):
    num_feat = train_envs[0]['features'].shape[1]
    print('training on', len(train_envs), 'environments (using', num_feat, 'features):')
    for e in train_envs:
        print('   ', e['info'], 'with', len(e['labels']), 'samples')
    print('\ntesting on:')
    for e in test_envs:
        print('   ', e['info'], 'with', len(e['labels']), 'samples')



In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import torch

YEARS = [2014, 2015, 2016, 2017, 2018]

class ToNum:
    def __init__(self):
        self.symbols = []

    def convert(self, symbol):
        if not symbol in self.symbols:
            self.symbols.append(symbol)
            return len(self.symbols) - 1
        else:
            return self.symbols.index(symbol)

    def index(self, key):
        return self.symbols.index(key)


def get_years(years=[], anchors=[]):
    device = set_device()

    # Get the full data across training and testing environments
    data_all = [pd.read_csv('data/%s_clean.csv' % y, index_col=0) for y in YEARS]

    k = data_all[0].keys()
    for d in data_all:
        k = k.intersection(d.keys())
    #print(k)

    data_all = [d[k] for d in data_all]
    # stack everything
    _data = None
    for d in data_all:
        if _data is None:
            _data = d
        else:
            _data = pd.concat([_data, d])
    data_all = _data

    # Yahuan Zheng: Build the sector mapping
    sectors = data_all['Sector'].unique()
    le = preprocessing.LabelEncoder()
    sector_labels = le.fit_transform(sectors)
    sector_mapping = dict(zip(le.classes_, range(len(le.classes_))))

    # Yahuan Zheng: Get the training data and add an additional column for sector labelling
    data = [pd.read_csv('data/%s_clean.csv' % y, index_col=0) for y in years]
    data = [pd.concat([d[k], d.iloc[:,-1:]], axis = 1) for d in data]  # Yahuan Zheng: IMPORTANT!!! Add the final column consisting of annual returns (otherwise discarded)

    for d in data:
        d['Sector_label'] = d['Sector'].apply(lambda x: sector_mapping[x])

    k_data = k.drop(['Sector', 'Class'])   # Sector_label is not included in k in the first place
    k_target = 'Class'

    # Yahuan Zheng: Shuffle the data
    data_shuffled = []
    for d in data:
        num_0 = d['Class'].isin([0]).sum()
        num_1 = d['Class'].isin([1]).sum()
        n = min(num_0, num_1)
        class_0 = d.nsmallest(n, 'Class')
        class_1 = d.nlargest(n, 'Class')
        d_shuffled = pd.concat([class_0, class_1]).sample(frac=1)
        #d_shuffled = class_0.append(class_1).sample(frac=1)
        data_shuffled.append(d_shuffled)    # returns a list of dataframes for the desired years
    data = data_shuffled

    x = [d[k_data].to_numpy() for d in data]
    y = [d[k_target].to_numpy() for d in data]
    x_anchors = [d[anchors].to_numpy() for d in data]   # Yahuan Zheng: add the anchor variables
    r = [d.iloc[:,-2:-1].to_numpy() for d in data]        # Yahuan Zheng: add the annual returns


    return [{
        'features': torch.tensor(x_, dtype=torch.float32).to(device),
        'labels': torch.tensor(y_.reshape(-1, 1), dtype=torch.float32).to(device),
        'anchors': torch.tensor(a_, dtype=torch.float32).to(device),
        'returns': torch.tensor(r_, dtype=torch.float32).to(device),
        'info': yr_,
        } for x_, y_, a_, r_, yr_ in zip(x, y, x_anchors, r, years)]


def get_sectors():
    device = set_device()
    data_all = [pd.read_csv('data/%s_clean.csv' % y, index_col=0) for y in YEARS]

    k = data_all[0].keys()
    for d in data_all:
        k = k.intersection(d.keys())

    data = [d[k] for d in data_all]

    # stack everything
    _data = None
    for d in data:
        if _data is None:
            _data = d
        else:
            _data = pd.concat([_data, d])
    data = _data

    # sort by sectors and shuffle the data within each sector
    sectors = data['Sector'].unique()
    _data = []
    for s in sectors:
        ids = data['Sector'].isin([s])
        _data = pd.concat([_data, data[ids].sample(frac=1)])
    data = _data

    data_shuffled = []
    for d in data:
        num_0 = d['Class'].isin([0]).sum()
        num_1 = d['Class'].isin([1]).sum()
        n = min(num_0, num_1)
        class_0 = d.nsmallest(n, 'Class')
        class_1 = d.nlargest(n, 'Class')
        d_shuffled = pd.concat([class_0, class_1 ]).sample(frac=1)
        data_shuffled.append(d_shuffled)
    data = data_shuffled

    k_data = k.drop(['Sector', 'Class'])
    k_target = 'Class'

    x = [d[k_data].to_numpy() for d in data]
    y = [d[k_target].to_numpy() for d in data]

    return [{
        'features': torch.tensor(x_, dtype=torch.float32).to(device),
        'labels': torch.tensor(y_.reshape(-1, 1), dtype=torch.float32).to(device),
        'info': s_
        } for x_, y_, s_ in zip(x, y, sectors)]

# if __name__ == '__main__':

#    x, y = get_envs()


# For ERM, IRM and REx:

In [ ]:
# Use CLASS LABEL as the target variable
# Run ERM, IRM and REx

import argparse
import numpy as np
import numpy.lib as npl
import torch
import matplotlib.pyplot as plt
from torch import nn, optim, autograd
from sklearn.linear_model import LinearRegression
#from load import get_years, get_sectors, YEARS
#from utils import *

parser = argparse.ArgumentParser(description='Finance Experiment')
parser.add_argument('--hidden_dim', type=int, default=256)
parser.add_argument('--l2_regularizer_weight', type=float,default=0.001)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--n_restarts', type=int, default=10)
parser.add_argument('--penalty_anneal_iters', type=int, default=100)
parser.add_argument('--irm_penalty_weight', type=float, default=0.0)
parser.add_argument('--rex_penalty_weight', type=float, default=10000.0)      # Can adjust the IRM and REx penalty weight to choose which method to perform. Setting both to 0 equals ERM.
parser.add_argument('--steps', type=int, default=501)
parser.add_argument('--plot', action='store_true')
parser.add_argument('--save', type=str, default='')
parser.add_argument('--train_envs', type=str, default='2014,2015,2016')
parser.add_argument('--test_envs', type=str, default='')
flags, unknown = parser.parse_known_args()

train_env_ids = [int(s.strip()) for s in flags.train_envs.split(',')]
if flags.test_envs:
    test_env_ids = [int(s.strip()) for s in flags.test_envs.split(',')]
else:
    test_env_ids = npl.setxor1d(YEARS, train_env_ids)


print('Flags:')
for k,v in sorted(vars(flags).items()):
    print("    {}: {}".format(k, v))


def whiten(x):
    with torch.no_grad():
        x -= x.mean(dim=0)
        x /= x.std(dim=0)
    return x

def mean_nll(logits, y):
    return nn.functional.binary_cross_entropy_with_logits(logits, y)

def mean_accuracy(logits, y):
    preds = (logits > 0.).float()
    return ((preds - y).abs() < 1e-2).float().mean()

def env_irm_penalty(logits, y):
    device = set_device()
    scale = torch.tensor(1.).to(device).requires_grad_()
    loss = mean_nll(logits * scale, y)
    grad = autograd.grad(loss, [scale], create_graph=True)[0]
    return torch.mean(grad**2)

def get_rex_penalty(train_envs):
    losses = torch.stack([e['nll'] for e in train_envs])
    penalty = torch.var(losses)
    return penalty


class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        lin1 = nn.Linear(input_size, flags.hidden_dim)
        lin2 = nn.Linear(flags.hidden_dim, flags.hidden_dim)
        lin3 = nn.Linear(flags.hidden_dim, 1)
        for lin in [lin1, lin2, lin3]:
            nn.init.xavier_uniform_(lin.weight)
            nn.init.zeros_(lin.bias)
        self._main = nn.Sequential(
            lin1, nn.Tanh(), #nn.ReLU(True),
            nn.Dropout(),
            lin2, nn.Tanh(), #nn.ReLU(True),
            nn.Dropout(),
            lin3)

    def forward(self, x):
        x = x.view(x.shape[0], self.input_size)
        out = self._main(x)
        return out


final_train_accs = []
final_test_accs = []
logs = []
for restart in range(flags.n_restarts):
    print("\n-------------------------------------- Restart", restart+1, "--------------------------------------\n")

    train_envs = get_years(train_env_ids)
    test_envs = get_years(test_env_ids)
    # preprocess
    for e in train_envs + test_envs:
        e['features'] = whiten(e['features'])
        e['anchors'] = whiten(e['anchors'])
    print_env_info(train_envs, test_envs)

    # init
    device = set_device()
    logger = Logger()
    mlp = MLP(train_envs[0]['features'].shape[1]).to(device)
    optimizer = optim.Adam(mlp.parameters(), lr=flags.lr)

    pretty_print('\n\nstep', 'train nll', 'train acc', 'irm penalty', 'rex penalty', 'test acc')

    for step in range(flags.steps):
        for env in train_envs + test_envs:
            env['logits'] = mlp(env['features'])
            env['nll'] = mean_nll(env['logits'], env['labels'])
            env['acc'] = mean_accuracy(env['logits'], env['labels'])
            env['penalty'] = env_irm_penalty(env['logits'], env['labels'])

        train_nll = torch.stack([e['nll'] for e in train_envs]).mean()
        train_acc = torch.stack([e['acc'] for e in train_envs]).mean()
        irm_penalty = torch.stack([e['penalty'] for e in train_envs]).mean()
        rex_penalty = get_rex_penalty(train_envs)

        weight_norm = torch.tensor(0.).to(device)
        for w in mlp.parameters():
            weight_norm += w.norm().pow(2)

        loss = train_nll.clone()
        loss += flags.l2_regularizer_weight * weight_norm


        if flags.irm_penalty_weight:
            if step >= flags.penalty_anneal_iters:
                loss /= flags.irm_penalty_weight
            loss += irm_penalty

        elif flags.rex_penalty_weight:
            if step >= flags.penalty_anneal_iters:
                loss /= flags.rex_penalty_weight
            loss += rex_penalty


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logger.log('train_nll', train_nll)
        logger.log('train_acc', train_acc)
        logger.log('irm_penalty', irm_penalty)
        logger.log('rex_penalty', rex_penalty)
        logger.log('test_acc', [e['acc'] for e in test_envs])
        logger.log('losses', [e['nll'] for e in train_envs])

        if step % 100 == 0:
            print_stats(step, logger)

    final_train_accs.append(np.mean(logger['train_acc'][-50:]))
    final_test_accs.append(np.mean(logger['test_acc'][-50:]))
    print('\n\nFinal train acc (mean/std across restarts so far):')
    print(np.mean(final_train_accs), np.std(final_train_accs))
    print('\nFinal test acc (mean/std across restarts so far):')
    print(np.mean(final_test_accs), np.std(final_test_accs))

    logs.append(logger)

    if flags.plot:
        plot(logger)

if flags.save:
    save(logs, 'results/%s_%s_%s' % ('', ','.join([str(e) for e in train_env_ids]), ','.join([str(e) for e in test_env_ids])))



In [ ]:
# Use ANNUAL RETURN as the target variable
# Run ERM, IRM and REx

import argparse
import numpy as np
import numpy.lib as npl
import torch
import matplotlib.pyplot as plt
from torch import nn, optim, autograd
from sklearn.linear_model import LinearRegression
#from load import get_years, get_sectors, YEARS
#from utils import *


parser = argparse.ArgumentParser(description='Finance Experiment')
parser.add_argument('--hidden_dim', type=int, default=256)
parser.add_argument('--l2_regularizer_weight', type=float,default=0.001)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--n_restarts', type=int, default=10)
parser.add_argument('--penalty_anneal_iters', type=int, default=100)
parser.add_argument('--irm_penalty_weight', type=float, default=0.0)
parser.add_argument('--rex_penalty_weight', type=float, default=10000.0)      # Can adjust the IRM and REx penalty weight to choose which method to perform. Setting both to 0 equals ERM
parser.add_argument('--steps', type=int, default=501)
parser.add_argument('--plot', action='store_true')
parser.add_argument('--save', type=str, default='')
parser.add_argument('--train_envs', type=str, default='2014,2015,2016')
parser.add_argument('--test_envs', type=str, default='')
flags, unknown = parser.parse_known_args()

train_env_ids = [int(s.strip()) for s in flags.train_envs.split(',')]
if flags.test_envs:
    test_env_ids = [int(s.strip()) for s in flags.test_envs.split(',')]
else:
    test_env_ids = npl.setxor1d(YEARS, train_env_ids)


print('Flags:')
for k,v in sorted(vars(flags).items()):
    print("    {}: {}".format(k, v))


def whiten(x):
    with torch.no_grad():
        x -= x.mean(dim=0)
        x /= x.std(dim=0)
    return x


def env_irm_penalty(y_hat, y):
    device = set_device()
    scale = torch.tensor(1.).to(device).requires_grad_()
    loss = torch.nn.functional.mse_loss(y_hat * scale, y)
    grad = autograd.grad(loss, [scale], create_graph=True)[0]
    return torch.mean(grad**2)

def get_rex_penalty(train_envs):
    losses = torch.stack([e['MSE'] for e in train_envs])      # Yahuan Zheng: Changed to MSE to use annual returns as target variable
    penalty = torch.var(losses)
    return penalty


class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        lin1 = nn.Linear(input_size, flags.hidden_dim)
        lin2 = nn.Linear(flags.hidden_dim, flags.hidden_dim)
        lin3 = nn.Linear(flags.hidden_dim, 1)
        for lin in [lin1, lin2, lin3]:
            nn.init.xavier_uniform_(lin.weight)
            nn.init.zeros_(lin.bias)
        self._main = nn.Sequential(
            lin1, nn.Tanh(), #nn.ReLU(True),
            nn.Dropout(),
            lin2, nn.Tanh(), #nn.ReLU(True),
            nn.Dropout(),
            lin3)

    def forward(self, x):
        x = x.view(x.shape[0], self.input_size)
        out = self._main(x)
        return out

final_train_mse = []
final_test_mse = []
final_train_acc = []
final_test_acc = []
logs = []

for restart in range(flags.n_restarts):
    print("\n-------------------------------------- Restart", restart+1, "--------------------------------------\n")

    train_envs = get_years(train_env_ids)
    test_envs = get_years(test_env_ids)
    # preprocess
    for e in train_envs + test_envs:
        e['features'] = whiten(e['features'])
        e['anchors'] = whiten(e['anchors'])
        e['returns'] = whiten(e['returns'])
    print_env_info(train_envs, test_envs)

    # init
    device = set_device()
    logger = Logger()
    mlp = MLP(train_envs[0]['features'].shape[1]).to(device)
    optimizer = optim.Adam(mlp.parameters(), lr=flags.lr)

    pretty_print('\n\nstep', 'train mse', 'train acc', 'irm penalty', 'rex penalty', 'test mse', 'test acc')

    for step in range(flags.steps):
        for env in train_envs + test_envs:
            env['preds'] = mlp(env['features'])     # Predict the annual return
            env['preds_labels'] = (env['preds'] > 0.).float()   # Assign the label based on the predicted return
            env['MSE'] = torch.nn.functional.mse_loss(env['preds'], env['returns'])
            env['acc'] = ((env['preds_labels'] - env['labels']).abs() < 1e-2).float().mean()
            env['penalty'] = env_irm_penalty(env['preds'], env['returns'])

        train_mse = torch.stack([e['MSE'] for e in train_envs]).mean()
        train_acc = torch.stack([e['acc'] for e in train_envs]).mean()
        irm_penalty = torch.stack([e['penalty'] for e in train_envs]).mean()
        rex_penalty = get_rex_penalty(train_envs)

        weight_norm = torch.tensor(0.).to(device)
        for w in mlp.parameters():
            weight_norm += w.norm().pow(2)

        loss = train_mse.clone()  # Yahuan Zheng: changed to MSE to use annual return as target variable
        loss += flags.l2_regularizer_weight * weight_norm


        if flags.irm_penalty_weight:
            if step >= flags.penalty_anneal_iters:
                loss /= flags.irm_penalty_weight
            loss += irm_penalty

        elif flags.rex_penalty_weight:
            if step >= flags.penalty_anneal_iters:
                loss /= flags.rex_penalty_weight
            loss += rex_penalty


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logger.log('train_mse', train_mse)
        logger.log('train_acc', train_acc)
        logger.log('irm_penalty', irm_penalty)
        logger.log('rex_penalty', rex_penalty)
        logger.log('test_mse', [e['MSE'] for e in test_envs])
        logger.log('test_acc', [e['acc'] for e in test_envs])

        if step % 100 == 0:
            pretty_print(
                          np.int32(step),
                          np.mean(logger['train_mse'][-50:]),
                          np.mean(logger['train_acc'][-50:]),
                          np.mean(logger['irm_penalty'][-50:]),
                          np.mean(logger['rex_penalty'][-50:]),
                          np.mean(logger['test_mse'][-50:]),
                          np.mean(logger['test_acc'][-50:])
                      )

    final_train_mse.append(np.mean(logger['train_mse'][-50:]))
    final_test_mse.append(np.mean(logger['test_mse'][-50:]))
    final_train_acc.append(np.mean(logger['train_acc'][-50:]))
    final_test_acc.append(np.mean(logger['test_acc'][-50:]))

    print('\n\nFinal train mse (mean/std across restarts so far):')
    print(np.mean(final_train_mse), np.std(final_train_mse))
    print('\nFinal train acc (mean/std across restarts so far):')
    print(np.mean(final_train_acc), np.std(final_train_acc))
    print('\nFinal test mse (mean/std across restarts so far):')
    print(np.mean(final_test_mse), np.std(final_test_mse))
    print('\nFinal test acc (mean/std across restarts so far):')
    print(np.mean(final_test_acc), np.std(final_test_acc))

    logs.append(logger)

    if flags.plot:
        plot(logger)

if flags.save:
    save(logs, 'results/%s_%s_%s' % ('', ','.join([str(e) for e in train_env_ids]), ','.join([str(e) for e in test_env_ids])))



Flags:
    hidden_dim: 256
    irm_penalty_weight: 0.0
    l2_regularizer_weight: 0.001
    lr: 0.001
    n_restarts: 10
    penalty_anneal_iters: 100
    plot: False
    rex_penalty_weight: 10000.0
    save: 
    steps: 501
    test_envs: 
    train_envs: 2014,2015,2016

-------------------------------------- Restart 1 --------------------------------------

training on 3 environments (using 37 features):
    2014 with 3228 samples
    2015 with 2458 samples
    2016 with 3158 samples

testing on:
    2017 with 2736 samples
    2018 with 2692 samples


step          train mse       train acc       irm penalty     rex penalty     test mse        test acc     
0               1.59305         0.49158         1.33458         0.01011         1.57808         0.50429      
100             1.02274         0.54400         0.01130         0.00092         1.03560         0.52933      
200             1.01060         0.53222         0.00333         5.85353e-05     1.02505         0.52637      
30

# For anchor regression:

In [ ]:
# Yahuan Zheng: Define the function to perform anchor transformation

def anchor_transform(x, y, x_anchor, gamma):
    '''
      x: a dataframe of input data
      y: a dataframe of labels
      x_anchor: a dataframe representing the data of the anchors
      gamma: a scalar regularization parameter
    '''
    device = set_device()
    n_samples = len(x)
    dim = len(x[0])
    id = torch.eye(n_samples).to(device)

    anchor_data = torch.Tensor(x_anchor).to(device)
    a_t_a = torch.t(anchor_data) @ anchor_data
    a_t_a_inv = torch.linalg.inv(a_t_a)
    proj_mat = (anchor_data @ a_t_a_inv) @ torch.t(anchor_data)

    left_x = torch.sub(id, proj_mat) @ x
    left_y = torch.sub(id, proj_mat) @ y
    right_x = torch.mul(proj_mat @ x, np.sqrt(gamma))
    right_y = torch.mul(proj_mat @ y, np.sqrt(gamma))

    x_trans = torch.add(left_x, right_x)
    y_trans = torch.add(left_y, right_y)

    return x_trans, y_trans

In [ ]:
# Yahuan Zheng: Anchor regression 1st try (brute force way)
# Use CLASS LABEL as the target variable

train_envs = get_years(train_env_ids, anchors=['Sector_label'])     # Yahuan Zheng: choose the anchors here
test_envs = get_years(test_env_ids)
regs = np.linspace(0, 20, num=201, endpoint=True)
best_reg = 1e6
best_acc = 0.0

# preprocess
for e in train_envs + test_envs:
  e['features'] = whiten(e['features'])
  e['anchors'] = whiten(e['anchors'])

pretty_print('gamma', 'train_acc', 'test_acc')

for gamma in regs:
    x_train = []
    y_train = []
    x_train_trans = []
    y_train_trans = []
    x_test = []
    y_test = []

    for e in train_envs:
      x, y = e['features'], e['labels']     # Using CLASS LABEL as target variable
      x_trans, y_trans = anchor_transform(e['features'], e['labels'], e['anchors'], gamma=gamma)
      x_train.append(x.cpu().numpy())
      y_train.append(y.cpu().numpy())
      x_train_trans.append(x_trans.cpu().numpy())
      y_train_trans.append(y_trans.cpu().numpy())

    for e in test_envs:
      x_test.append(e['features'].cpu().numpy())
      y_test.append(e['labels'].cpu().numpy())

    x_train = np.vstack(x_train)
    y_train = np.vstack(y_train)
    x_train_trans = np.vstack(x_train_trans)
    y_train_trans = np.vstack(y_train_trans)
    x_test = np.vstack(x_test)
    y_test = np.vstack(y_test)

    anchor_reg = LinearRegression(fit_intercept=False).fit(x_train_trans, y_train_trans)
    w = anchor_reg.coef_
    y_hat = anchor_reg.predict(x_test)
    y_hat_label = (y_hat > 0.5).astype(float)      # Predict the labels based on a threshold value of 0.5
    #train_mse = mean_squared_error(y_train, anchor_reg.predict(x_train))
    #test_mse = mean_squared_error(y_test, anchor_reg.predict(x_test))
    train_acc = (abs((y_train > 0.).astype(float) - (anchor_reg.predict(x_train) > 0.5).astype(float)) < 1e-2).astype(float).mean()
    test_acc = (abs(y_hat_label - (y_test > 0.).astype(float))< 1e-2).astype(float).mean()

    pretty_print(gamma, train_acc, test_acc)
    #print('--- Anchor regression with gamma = {:.1f} has accuracy {:.5f}.'.format(gamma, anchor_acc_test))

    '''
    if gamma % 10 == 0:
        plt.hist(y_hat, bins='auto', density=True)
        plt.title('Histogram of y_hat for Anchor regression with gamma={:.1f}'.format(gamma))
        plt.show()
    '''

    if test_acc > best_acc:
        best_reg = gamma
        best_acc = test_acc

print('\nBest reg is gamma = {:.1f} with {:.5f} accuracy.'.format(best_reg, best_acc))



In [ ]:
# Yahuan Zheng: Anchor regression 2nd try
# Use ANNUAL RETURN as the target variable

train_envs = get_years(train_env_ids, anchors=['Sector_label'])     # Yahuan Zheng: choose the anchors here
test_envs = get_years(test_env_ids)
regs = np.linspace(0, 20, num=201, endpoint=True)
best_reg = 1e6
best_acc = 0.0

# preprocess
for e in train_envs + test_envs:
  e['features'] = whiten(e['features'])
  e['anchors'] = whiten(e['anchors'])
  e['returns'] = whiten(e['returns'])

pretty_print('gamma', 'train mse', 'train_acc', 'test mse', 'test_acc')

for gamma in regs:
    x_train = []
    y_train = []
    x_train_trans = []
    y_train_trans = []
    x_test = []
    y_test = []

    for e in train_envs:
      x, y = e['features'], e['returns']
      x_trans, y_trans = anchor_transform(e['features'], e['returns'], e['anchors'], gamma=gamma)
      x_train.append(x.cpu().numpy())
      y_train.append(y.cpu().numpy())
      x_train_trans.append(x_trans.cpu().numpy())
      y_train_trans.append(y_trans.cpu().numpy())

    for e in test_envs:
      x_test.append(e['features'].cpu().numpy())
      y_test.append(e['returns'].cpu().numpy())

    x_train = np.vstack(x_train)
    y_train = np.vstack(y_train)
    x_train_trans = np.vstack(x_train_trans)
    y_train_trans = np.vstack(y_train_trans)
    x_test = np.vstack(x_test)
    y_test = np.vstack(y_test)

    anchor_reg = LinearRegression(fit_intercept=False).fit(x_train_trans, y_train_trans)
    w = anchor_reg.coef_
    y_hat = anchor_reg.predict(x_test)
    y_hat_label = (y_hat > 0.).astype(float)
    train_mse = mean_squared_error(y_train, anchor_reg.predict(x_train))
    test_mse = mean_squared_error(y_test, anchor_reg.predict(x_test))
    train_acc = (abs((y_train > 0.).astype(float) - (anchor_reg.predict(x_train) > 0.).astype(float)) < 1e-2).astype(float).mean()
    test_acc = (abs(y_hat_label - (y_test > 0.).astype(float))< 1e-2).astype(float).mean()

    pretty_print(gamma, train_mse, train_acc, test_mse, test_acc)
    #print('--- Anchor regression with gamma = {:.1f} has accuracy {:.5f}.'.format(gamma, anchor_acc_test))

    '''
    if gamma % 10 == 0:
        plt.hist(y_hat, bins='auto', density=True)
        plt.title('Histogram of y_hat for Anchor regression with gamma={:.1f}'.format(gamma))
        plt.show()
    '''

    if test_acc > best_acc:
        best_reg = gamma
        best_acc = test_acc

print('\nBest reg is gamma = {:.1f} with {:.5f} accuracy.'.format(best_reg, best_acc))



In [ ]:
### Optional (applicable for command line executions)

import matplotlib.pyplot as plt
import numpy as np
import numpy.lib as npl
import sys
#from load import YEARS

EXP_LIST = [[2014, 2015, 2016]]
"""
EXP_LIST = [
    [2014, 2015, 2016],
    [2014, 2015, 2017],
    [2014, 2015, 2018],
    [2014, 2016, 2017],
    [2014, 2016, 2018],
    [2014, 2017, 2018],
    [2015, 2016, 2017],
    [2015, 2016, 2018],
    [2015, 2017, 2018],
    [2016, 2017, 2018],
]
"""

def plot_data(means, stds, descr, plot_every=8, title='', ylabel=''):
    fig, ax = plt.subplots(1, figsize=(4,3))
    for m, s, info in zip(means, stds, descr):
        num_el = len(m)
        x = np.arange(num_el)
        x = x[:-(num_el%plot_every)].reshape(-1,plot_every).mean(1)
        y = m[:-(num_el%plot_every)].reshape(-1,plot_every).mean(1)
        dy = s[:-(num_el%plot_every)].reshape(-1,plot_every).mean(1)
        ax.fill_between(x, y-dy, y+dy, alpha=0.25)
        ax.plot(x, y, label=info)
        ax.set_title(title)
        ax.set_ylabel(ylabel)
        ax.set_xlabel("Iteration")

    fig.tight_layout()
    plt.legend()


if __name__ == "__main__":

    if sys.argv[-1] == 'all':
        # combine
        mat_erm, mat_irm, mat_rex = [np.loadtxt('results/%s_matrix.dat' % k) for k in ['erm', 'irm', 'rex']]
        mat_train = mat_erm[:, :len(YEARS)] / mat_erm[:, :len(YEARS)]
        mat_erm = mat_erm[:, len(YEARS):]
        mat_irm = mat_irm[:, len(YEARS):]
        mat_rex = mat_rex[:, len(YEARS):]

        irm_rel = 100 * (mat_irm - mat_erm) / mat_erm
        rex_rel = 100 * (mat_rex - mat_erm) / mat_erm

        notnan = irm_rel > -999
        vmax = np.abs([irm_rel[notnan].min(), irm_rel[notnan].max(), rex_rel[notnan].min(), rex_rel[notnan].max()]).max()

        e = mat_erm[notnan]
        i = mat_irm[notnan]
        r = mat_rex[notnan]
        ir = irm_rel[notnan]
        rr = rex_rel[notnan]

        print('cases irm better', (ir > 0).mean())
        print('cases rex better', (rr > 0).mean())
        print('irm rel performance (mean, std, min, max)', ir.mean(), ir.std(), ir.min(), ir.max())
        print('rex rel performance (mean, std, min, max)', rr.mean(), rr.std(), rr.min(), rr.max())
        print('erm performance (mean, std, min, max)', e.mean(), e.std(), e.min(), e.max())
        print('irm performance (mean, std, min, max)', i.mean(), i.std(), i.min(), i.max())
        print('rex performance (mean, std, min, max)', r.mean(), r.std(), r.min(), r.max())

        x, y = np.meshgrid(range(5), range(10))
        x = x[mat_train == 1]
        y = y[mat_train == 1]

        ax = plt.subplot(131)
        plt.grid(zorder=-99)
        plt.imshow(mat_train, cmap='binary')
        plt.scatter(x, y, c='k', zorder=10)
        plt.xticks(range(len(YEARS)), YEARS)
        plt.yticks(range(10), ['']*10)
        plt.title('Training envs.')
        plt.ylabel('Task')
        plt.xticks(rotation=45)
        plt.xlim(-0.5, 4.5)
        plt.ylim(9.5, -0.5)

        ax = plt.subplot(132)
        plt.imshow(irm_rel, cmap='PiYG', vmin=-vmax, vmax=vmax, zorder=1)
        plt.scatter(x, y, c='k', zorder=10)
        plt.xticks(range(len(YEARS)), YEARS)
        plt.yticks(range(10), ['']*10)
        plt.title('Test IRM')
        plt.xticks(rotation=45)

        ax = plt.subplot(133)
        im = plt.imshow(rex_rel, cmap='PiYG', vmin=-vmax, vmax=vmax, zorder=1)
        plt.scatter(x, y, c='k', zorder=10)
        plt.xticks(range(len(YEARS)), YEARS)
        plt.yticks(range(10), ['']*10)
        plt.title('Test REx')
        plt.xticks(rotation=45)


        cbar = plt.colorbar(im)
        cbar.ax.set_ylabel('Performance relative to ERM (%)', rotation=90)

        plt.savefig('grid.pdf')
        plt.show()
        exit()

    # plot single experiment
    train_envs = [np.array(e, dtype=int) for e in EXP_LIST]
    test_envs = [npl.setxor1d(YEARS, e) for e in train_envs]

    fnames = ['results/%s_%s_%s.npz' % ('',
        ','.join([str(e) for e in train_env]),
        ','.join([str(e) for e in test_env])) \
            for train_env, test_env in zip(train_envs, test_envs)]
    print(fnames)

    files = [np.load(fname) for fname in fnames]
    SMOOTH = 50

    train_means = [f['train_acc'][:,-SMOOTH:].mean() for f in files]
    train_stds = [f['train_acc'][:,-SMOOTH:].std() for f in files]

    max_pts = [f['test_acc'].mean(0)[:-1].reshape((-1, SMOOTH, 2)).mean(1).argmax(0).clip(1, np.inf) * SMOOTH for f in files]
    max_pts = [p.astype(int)[::-1] for p in max_pts]

    print('stopping points:', max_pts)

    test_means = [
        np.stack([
        f['test_acc'][:, pts[i]-SMOOTH//2:pts[i]+SMOOTH//2, i].mean() \
            for i in [0,1]]) \
                for f,pts in zip(files, max_pts)]

    out_img = np.zeros((len(train_envs), 2*len(YEARS)))

    for i, (e_tr, m_tr, e_te, m_te) in enumerate(zip(train_envs, train_means, test_envs, test_means)):
        out_img[i, e_tr-YEARS[0]] = m_tr
        out_img[i, len(YEARS)-YEARS[0]+e_te] = m_te

    np.savetxt('results/matrix.dat', out_img)
    #np.savetxt('results/%s_matrix.dat' % sys.argv[-1], out_img)
    plt.imshow(out_img)
    plt.show()

